# Pass arguments

To make your programs useful, you need to give them some arguments. On this page I'll describe how to do that.

## Preparing

To run the examples on this page, you will need:

- Start a docker container with `fastapi` (check [run application](../fastapi.ipynb#run-application));
- Import some libraries.

In [21]:
# requests is cnetral library
# to try make requests from python
import requests

!docker run --rm -itd\
    --name test_container\
    -v ./pass_arguments_files/app.py:/app.py\
    -p 8000:8000 \
    fastapi_experiment \
    uvicorn --host 0.0.0.0 --reload app:app

61bb7608cbaf6ae7dd722e4c8a3268e88c47a1d85248522fbfb332fb1a20f7ca


Don't forget to stop the container when you've finished playing with the examples on this page.

In [18]:
!docker stop test_container

test_container


## Syntax options

### Query params

To define url with parameters in `fastapi`, you need to define method with parameters and wrap it with `fastapi` object decorators.

In order to pass an argument using ulr we need to write a construction `?param1=argument1&param2=argument2&...&paramN=argumentN` at the end of url (in web development, the name for this construction is query params).

So in the following example I have written a program to divide two numbers and use the syntax `/divide?a=10&b=2` in the url to complete the division.

In [2]:
%%writefile pass_arguments_files/app.py
from fastapi import FastAPI

app = FastAPI()

@app.get("/divide")
def divide(a:int, b:int) -> int:
    return a/b

Overwriting pass_arguments_files/app.py


In [3]:
response = requests.get("http://localhost:8000/divide?a=10&b=2")
response.text

'5'

### Path params

You can specify some parameters as part of your url. Just specify url in fastapi decorators using the syntax `<path>/{param1}/{param2}/...`. So you can request this api by using the corresponding syntax.

In the next cell, we'll define a new application that will use the path param syntax for division api.

In [4]:
%%writefile pass_arguments_files/app.py
from fastapi import FastAPI

app = FastAPI()

@app.get("/divide/{a}/{b}")
def divide(a:int, b:int) -> float:
    return a/b

Overwriting pass_arguments_files/app.py


And here is example how it can be called:

In [5]:
response = requests.get("http://localhost:8000/divide/10/2")
response.text

'5.0'

### Pydantic model

You can define pydentic model as an input to your endpoint - simply by declaring pydentic model as a type in parameters.

So the following `fastapi` program defines `Item` which expects `json` with keys `param1` and `param2` and just returns line describing what data we've got.

In [6]:
%%writefile pass_arguments_files/app.py
from fastapi import FastAPI
from pydantic import BaseModel

class Item(BaseModel):
    param1: int
    param2: str

app = FastAPI()

@app.post("/")
def read_json(item:Item):
    return f"""I have got:
    param1={item.param1};
    param2={item.param2}."""

Overwriting pass_arguments_files/app.py


And here is an example of how you can send a query to such an end point and process its result using python `requests` library. Actually you just need to pass a json file of the appropriate form.

In [7]:
data = {"param1" : 2, "param2": "test line"}
response = requests.post(
    "http://localhost:8000/", 
    json=data
)
print(response.content.decode("utf-8").replace("\\n", "\n"))

"I have got:
    param1=2;
    param2=test line."


## Data types

You need to declare data types for the arguments otherwise the call will not work correctly.

The following example describes a programme without input datatypes. The request to the server causes the error:

In [23]:
%%writefile pass_arguments_files/app.py
from fastapi import FastAPI

app = FastAPI()

@app.get("/divide")
def divide(a, b) -> int:
    return f"{a} {b}"

Overwriting pass_arguments_files/app.py


In [24]:
response = requests.get("http://localhost:8000/divide?a=10&b=2")
response.text

'Internal Server Error'

### Datetime input

You are allowed to use `datetime.datetime` as input type in fastapi. But note that you have to use ISO 8601 fromat for datetime objects - `<YYYY-MM-DDTHH-MM-SS>`.
Where:

- YYYYY - year (e.g. 2022);
- MM - month (from 01 to 12);
- DD - day (from 01 to 31);
- T - separator between date and time;
- HH - hour (00 to 23);
- MM - minutes (from 00 to 59);
- SS - seconds (00 to 59).

So in the following example is the service that takes any date and returns the date for the next day:

In [25]:
%%writefile pass_arguments_files/app.py
from fastapi import FastAPI
from datetime import datetime, timedelta

app = FastAPI()

@app.get("/add_year")
def add_year(dt : datetime):
    return dt + timedelta(days = 1)

Overwriting pass_arguments_files/app.py


In [26]:
response = requests.get(
    "http://localhost:8000/add_year?dt=2022-10-05T20:10:10"
)
response.text

'"2022-10-06T20:10:10"'